<a href="https://colab.research.google.com/github/Xtinc-T/html-portfolio/blob/main/Optimisation_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Funtions :
<br>
function: 6*(x-4)^4+4*(y-4)^2
<br>
function: Max(x-4,0)+4*|y-4|



In [ ]:
from sympy import Symbol, Max, Derivative

# Define symbolic variables
x = Symbol('x')
y = Symbol('y')

# First function
function1 = Max(x-4, 0) + 4*abs(y-4)

# Differentiate function1 with respect to x (wrt x means w.r.t. x)
d_function1_dx = Derivative(function1, x).doit()

# Second function
function2 = 6*(x-4)**4 + 4*(y-4)**2

# Differentiate function2 with respect to x
d_function2_dx = Derivative(function2, x).doit()

# Print the results for x derivatives
print("Derivative of the first function with respect to x:", d_function1_dx)
print("Derivative of the second function with respect to x:", d_function2_dx)


Derivative of the first function with respect to x: Heaviside(x - 4)
Derivative of the second function with respect to x: 24*(x - 4)**3


(a) Your first task is to implement the following gradient descent updates that we studied
in the lectures: <br>
(i) Polyak step size <br>
(ii) RMSProp<br>
(iii) Heavy Ball <br>
(iv) Adam <br>


In [ ]:
def polyak_update(params, gradients, learning_rate, beta):
  """
  Updates model parameters using Polyak step size.

  Args:
      params: List of model parameters.
      gradients: List of gradients for each parameter.
      learning_rate: Base learning rate.
      beta: Polyak averaging coefficient (0 < beta < 1).

  Returns:
      List of updated model parameters.
  """
  updated_params = []
  velocity = [0.0] * len(params)  # Initialize velocity with zeros
  for i, param in enumerate(params):
    velocity[i] = beta * velocity[i] + (1 - beta) * gradients[i]
    updated_params.append(param - learning_rate * velocity[i])
  return updated_params

In [ ]:
def rmsprop_update(params, gradients, learning_rate, decay_rate, epsilon):
  """
  Updates model parameters using RMSProp.

  Args:
      params: List of model parameters.
      gradients: NumPy array of gradients for each parameter.
      learning_rate: Base learning rate.
      decay_rate: Decay rate for the moving average of squared gradients (0 < decay_rate < 1).
      epsilon: Small value for numerical stability (to avoid division by zero).

  Returns:
      List of updated model parameters.
  """
  updated_params = []
  cache = np.zeros_like(params)  # Initialize cache with zeros
  for i in range(len(params)):
    cache[i] = decay_rate * cache[i] + (1 - decay_rate) * gradients[i] ** 2
    updated_params.append(params[i] - learning_rate / (epsilon + np.sqrt(cache[i])) * gradients[i])
  return updated_params


In [ ]:
def heavy_ball_update(params, gradients, learning_rate, momentum):
  """
  Updates model parameters using Heavy Ball.

  Args:
      params: List of model parameters.
      gradients: List of gradients for each parameter.
      learning_rate: Base learning rate.
      momentum: Momentum coefficient (0 <= momentum < 1).

  Returns:
      List of updated model parameters.
  """
  updated_params = []
  velocity = [0.0] * len(params)  # Initialize velocity with zeros
  for i, param in enumerate(params):
    velocity[i] = momentum * velocity[i] + learning_rate * gradients[i]
    updated_params.append(param - velocity[i])
  return updated_params

In [ ]:
import numpy as np

def adam_update(params, gradients, learning_rate, beta1, beta2, epsilon):
  """
  Updates model parameters using Adam.

  Args:
      params: List of model parameters.
      gradients: List of gradients for each parameter.
      learning_rate: Base learning rate.
      beta1: Decay rate for the 1st moment estimate (0 < beta1 < 1).
      beta2: Decay rate for the 2nd moment estimate (0 < beta2 < 1).
      epsilon: Small value for numerical stability (to avoid division by zero).

  Returns:
      List of updated model parameters.
  """
  updated_params = []
  m = [0.0] * len(params)  # Initialize 1st moment estimate with zeros
  v = [0.0] * len(params)  # Initialize 2nd moment estimate with zeros
  t = 0  # Initialize time step

  for i, param in enumerate(params):
    t += 1
    m[i] = beta1 * m[i] + (1 - beta1) * gradients[i]
    v[i] = beta2 * v[i] + (1 - beta2) * gradients[i] ** 2
    m_hat = m[i] / (1 - beta1 ** t)
    v_hat = v[i] / (1 - beta2 ** t)
    updated_params.append(param - learning_rate * m_hat / (np.sqrt(v_hat)))


(b) Now apply these algorithms to each of the functions that you downloaded to show
the impact of changing the following parameters, and discuss:<br>
(i) α and β in RMSProp.<br>
(ii) α and β in Heavy Ball.<br>
(iii) α, β1 and β2 in Adam.<br>

In [ ]:
# Gradient of function1 with respect to x and y
d_function1_dx = Derivative(function1, x).doit()
d_function1_dy = Derivative(function1, y).doit()

# Gradient of function2 with respect to x and y
d_function2_dx = Derivative(function2, x).doit()
d_function2_dy = Derivative(function2, y).doit()


In [ ]:
# Define initial parameters for optimization
initial_params = [5, 5]  # Initial values for x and y

# Define gradients for each function
gradients_function1 = [d_function1_dx.subs({x: initial_params[0], y: initial_params[1]}),
                       d_function1_dy.subs({x: initial_params[0], y: initial_params[1]})]
gradients_function2 = [d_function2_dx.subs({x: initial_params[0], y: initial_params[1]}),
                       d_function2_dy.subs({x: initial_params[0], y: initial_params[1]})]

# Apply RMSProp with different parameters
# (i) α and β
# Let's vary α (learning_rate) and β (decay_rate)
learning_rates = [0.01, 0.1, 1]
decay_rates = [0.9, 0.99, 0.999]

for alpha in learning_rates:
  for beta in decay_rates:
    # Evaluate gradients numerically
    gradients_function1_numerical = gradients_function1.subs({x: initial_params[0], y: initial_params[1]}).evalf()
    gradients_function1_numerical = np.array(gradients_function1_numerical)  # Convert to NumPy array

    updated_params_rmsprop = rmsprop_update(initial_params, gradients_function1_numerical, alpha, beta, epsilon=1e)

# Apply Heavy Ball with different parameters
# (ii) α and β
# Let's vary α (learning_rate) and β (momentum)
learning_rates = [0.01, 0.1, 1]
momentum_values = [0.1, 0.5, 0.9]

for alpha in learning_rates:
    for beta in momentum_values:
        updated_params_heavy_ball = heavy_ball_update(initial_params, gradients_function1, alpha, beta)
        print(f"Heavy Ball with alpha={alpha} and beta={beta}: {updated_params_heavy_ball}")

# Apply Adam with different parameters
# (iii) α, β1, and β2
# Let's vary α (learning_rate), β1, and β2
learning_rates = [0.01, 0.1, 1]
beta1_values = [0.9, 0.99, 0.999]
beta2_values = [0.999, 0.9999, 0.99999]

for alpha in learning_rates:
    for beta1 in beta1_values:
        for beta2 in beta2_values:
            updated_params_adam = adam_update(initial_params, gradients_function1, alpha, beta1, beta2, epsilon=1e-8)
            print(f"Adam with alpha={alpha}, beta1={beta1}, and beta2={beta2}: {updated_params_adam}")


SyntaxError: invalid decimal literal (<ipython-input-27-4e6aa9200e51>, line 22)